In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Load the pre-trained MobileNet model (excluding the top classification layers)
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a custom classification head for your 10 classes
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

# Create a new model combining the base MobileNet and custom head
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
# Define data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Define data augmentation for validation/testing
val_datagen = ImageDataGenerator(rescale=1./255)

# Load your dataset (replace 'train_dir' and 'val_dir' with your dataset directories)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Tomato_Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Tomato_Val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)



Found 8950 images belonging to 10 classes.
Found 2295 images belonging to 10 classes.


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=25,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/25
280/280 [==============================] - 2620s 9s/step - loss: 0.9830 - accuracy: 0.6906 - val_loss: 0.5292 - val_accuracy: 0.8292
Epoch 2/25
280/280 [==============================] - 134s 477ms/step - loss: 0.4264 - accuracy: 0.8663 - val_loss: 0.3784 - val_accuracy: 0.8789
Epoch 3/25
280/280 [==============================] - 131s 470ms/step - loss: 0.3208 - accuracy: 0.8984 - val_loss: 0.2798 - val_accuracy: 0.9085
Epoch 4/25
280/280 [==============================] - 132s 469ms/step - loss: 0.2658 - accuracy: 0.9111 - val_loss: 0.2866 - val_accuracy: 0.9059
Epoch 5/25
280/280 [==============================] - 131s 470ms/step - loss: 0.2272 - accuracy: 0.9245 - val_loss: 0.2459 - val_accuracy: 0.9242
Epoch 6/25
280/280 [==============================] - 134s 477ms/step - loss: 0.2091 - accuracy: 0.9307 - val_loss: 0.2337 - val_accuracy: 0.9277
Epoch 7/25
280/280 [==============================] - 131s 466ms/step - loss: 0.1823 - accuracy: 0.9435 - val_loss: 0.2082 - v

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Models/Mobilenet_transfer_learning_model_1.h5')